In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import math
from IPython.display import display, Math
import scipy.stats as st
rcParams.update({'font.size': 12})

### Вариант 9

Пусть $$x_1, ..., x_n \sim Bin(m, p)$$

+ Покажите, что параметр 𝑝 попадает в доверительный интервал с нужной вероятностью.
+ Исследуйте зависимость ширины доверительного интервала для параметра распределения 𝑝 от объема выборки.

точечная оценка: $$\widehat{p}_n = \frac{\overline{x}}{m}$$

доверительный интервал уровня доверия 0.95:

Так как имеем n независимых одинаково распределенных случайных величин, и дисперсия распределения неизвестна, то выполняется:

$$\sqrt{n}\frac{\overline{x} - mp}{\tilde{s}_n} \rightarrow T_{n-1}$$


$$P(-t_{0.975}^{n-1} \leq \sqrt{n}\frac{\overline{x} - mp}{\tilde{s}_n} \leq t_{0.975}^{n-1}) \geq 0.95$$

$$P(-t_{0.975}^{n-1} \leq \sqrt{n} m \frac{\frac{\overline{x}}{m} - p}{\tilde{s}_n} \leq t_{0.975}^{n-1}) \geq 0.95$$


откуда можем найти границы доверительного интервала для нашей оценки параметра p:

$$[\frac{\overline{x}}{m} - t_{0.975} \frac{\tilde{s}_n}{m\sqrt{n}};\frac{\overline{x}}{m} + t_{0.975} \frac{\tilde{s}_n}{m\sqrt{n}}]$$


Для того, чтобы показать, что параметр 𝑝 попадает в доверительный интервал с нужной вероятностью, буду выбирать по 200 раз выборки из n элементов(n=50) из биномиального распределения с параметрами m и p, подсчитывать границы доверительного интервала и проверять, попадает ли параметр в этот отрезок. 

In [2]:
def calc_CI(m, p, n, quantile):
    t_q = st.t.ppf(q=quantile, df=n-1)
    sample = np.random.binomial(m, p, size = n)
    var_notbiased = np.var(sample, ddof=1)
    point_est = np.mean(sample)/m
    CI_left = point_est - t_q * (var_notbiased / (m * math.sqrt(n)))
    CI_right = point_est + t_q * (var_notbiased / (m * math.sqrt(n)))
    return CI_left, CI_right

In [3]:
k = 1000
n = 100
params = [[20, 0.3,  0.975], [30, 0.3,  0.975], [30, 0.5,  0.975], [30, 0.7, 0.975], [30, 0.3,  0.975]]

for i, par in enumerate(params):
    m, p, quantile = par
    CIs_true = []
    for _ in range(k):
        CI_left, CI_right = calc_CI(m, p, n, quantile)
        CIs_true.append(CI_left<= p <= CI_right)
    res = sum(CIs_true)/len(CIs_true)
    print(f'{i}: CI = [{CI_left},{CI_right}]; параметр попадает в {res*100} % случаях, p:{p}, m:{m}')

0: CI = [0.25548821556177986,0.3315117844382201]; параметр попадает в 100.0 % случаях, p:0.3, m:20
1: CI = [0.2520658384509505,0.3312674948823829]; параметр попадает в 99.9 % случаях, p:0.3, m:30
2: CI = [0.47452393661078907,0.5561427300558776]; параметр попадает в 100.0 % случаях, p:0.5, m:30
3: CI = [0.6546440248316848,0.7386893085016485]; параметр попадает в 100.0 % случаях, p:0.7, m:30
4: CI = [0.2593344079090692,0.3486655920909308]; параметр попадает в 100.0 % случаях, p:0.3, m:30


+ Исследуйте зависимость ширины доверительного интервала для параметра распределения 𝑝 от объема выборки.

Покажу на примере 95% доверительных интервалов для выборок из распределения с параметром m=100 и p=0.7, увеличивая n

In [4]:
m = 100
p = 0.7
quantile = 0.975
for n in [5, 10, 50, 100, 300, 500, 1000, 10000]:
    CI_left, CI_right = calc_CI(m, p, n, quantile)
    print(f'n: {n}, CI = [{CI_left},{CI_right}], длина интервала: {CI_right-CI_left}')

n: 5, CI = [0.34080915255144356,1.1031908474485563], длина интервала: 0.7623816948971127
n: 10, CI = [0.5292756501209874,0.8607243498790125], длина интервала: 0.33144869975802504
n: 50, CI = [0.6218357653605244,0.7601642346394755], длина интервала: 0.13832846927895104
n: 100, CI = [0.6557393120251865,0.7286606879748135], длина интервала: 0.072921375949627
n: 300, CI = [0.6742604861437762,0.7192061805228904], длина интервала: 0.04494569437911422
n: 500, CI = [0.6777282598065675,0.7133517401934326], длина интервала: 0.035623480386865136
n: 1000, CI = [0.6884357728196993,0.712424227180301], длина интервала: 0.023988454360601708
n: 10000, CI = [0.6962419648951673,0.7043680351048328], длина интервала: 0.008126070209665492


Как можно заметить, с ростом выборки длин доверительного интервала уменьшается